In [1]:
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import seaborn as sns
import pandas as pd
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
progress = ProgressBar().register()

In [4]:
IMG_DIR = 'analysis'
DPI=120

## Loading data using Dask (loads lazily)
* https://www.youtube.com/watch?v=RA_2qdipVng&t=1s
* http://matthewrocklin.com/slides/scipy-2017.html

In [7]:
# http://stat-computing.org/dataexpo/2009/the-data.html
# !curl -O http://stat-computing.org/dataexpo/2009/2000.csv.bz2
# !curl -O http://stat-computing.org/dataexpo/2009/2001.csv.bz2
# !curl -O http://stat-computing.org/dataexpo/2009/2002.csv.bz2
# !ls -lh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 78.7M  100 78.7M    0     0  4067k      0  0:00:19  0:00:19 --:--:-- 4658k78.7M   69 55.0M    0     0  3857k      0  0:00:20  0:00:14  0:00:06 4477k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 79.6M  100 79.6M    0     0  3725k      0  0:00:21  0:00:21 --:--:-- 4115k  0  2937k      0  0:00:27  0:00:07  0:00:20 4178k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 72.3M  100 72.3M    0     0  4530k      0  0:00:16  0:00:16 --:--:-- 5467k
total 116G
-rwxrwxrwx 1 root root  79M Oct  3 11:37 2000.csv.bz2
-rwxrwxrwx 1 root root  80M Oct  3 11:37 2001.csv.bz2
-rwxrwxrwx 1 root root  73M Oct  3 11:38 2002.

In [12]:
# !bzip2 -d 2000.csv.bz2
# !bzip2 -d 2001.csv.bz2
# !bzip2 -d 2002.csv.bz2
# !ls -lh

total 812G
-rwxrwxrwx 1 root root 544M Oct  3 11:37 2000.csv
-rwxrwxrwx 1 root root 573M Oct  3 11:37 2001.csv
-rwxrwxrwx 1 root root 506M Oct  3 11:38 2002.csv
-rwxrwxrwx 1 root root 365K Sep  7 15:54 analysis.ipynb
drwxrwxrwx 0 root root  512 Oct  3 11:37 data
-rwxrwxrwx 1 root root 9.1K Sep 28 11:37 elastic.ipynb
-rwxrwxrwx 1 root root 1.9K Sep 28 11:37 elasticsearch-py.ipynb
-rwxrwxrwx 1 root root  25K Oct  3 11:34 import.ipynb
-rwxrwxrwx 1 root root  27K Oct  3 11:40 unsupervised.ipynb


In [13]:
# !mkdir data
# !mv *.csv data

In [14]:
# data_types = {'CRSElapsedTime': int, 'CRSDepTime': int, 'Year': int, 'Month': int, 'DayOfWeek': int, 'DayofMonth': int}
data_types = {'CRSDepTime': int, 'Year': int, 'Month': int, 'DayOfWeek': int, 'DayofMonth': int}

# http://dask.pydata.org/en/latest/dataframe-overview.html
%time lazy_df = dd.read_csv('./data/*.csv', encoding='iso-8859-1', dtype=data_types, assume_missing=True)

CPU times: user 281 ms, sys: 188 ms, total: 469 ms
Wall time: 1.02 s


In [15]:
%time lazy_df.head()

[########################################] | 100% Completed |  3.8s
CPU times: user 2.59 s, sys: 1.05 s, total: 3.64 s
Wall time: 3.86 s


,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2000,1,28,5,1647.0,1647,1906.0,1859.0,HP,154.0,...,15.0,11.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
1,2000,1,29,6,1648.0,1647,1939.0,1859.0,HP,154.0,...,5.0,47.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2,2000,1,30,7,NaN,1647,NaN,1859.0,HP,154.0,...,0.0,0.0,1.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
3,2000,1,31,1,1645.0,1647,1852.0,1859.0,HP,154.0,...,7.0,14.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
4,2000,1,1,6,842.0,846,1057.0,1101.0,HP,609.0,...,3.0,8.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN


## Cleaning and fixing data

In [16]:
%time lazy_df = lazy_df.fillna(-1)

CPU times: user 0 ns, sys: 15.6 ms, total: 15.6 ms
Wall time: 42 ms


In [18]:
# 2400 is not a valid time
lazy_df['CRSDepTime'] = lazy_df.apply(lambda row: 2359 if row['CRSDepTime'] == 2400 else row['CRSDepTime'], axis='columns')

## Sampling to smaller dataset and further cleaning

In [20]:
# %time len(lazy_df)

[########################################] | 100% Completed | 23min  9.2s
CPU times: user 20min 24s, sys: 1min 54s, total: 22min 19s
Wall time: 23min 9s


16922186

In [23]:
# http://dask.pydata.org/en/latest/dataframe-api.html#dask.dataframe.DataFrame.sample
s = 10000  # desired sample size
n = 16922186
fraction = s / n
lazy_df = lazy_df.sample(fraction)

In [24]:
# first turn our 10000 samples into a normal pandas df for convenience
%time df = lazy_df.compute()

[########                                ] | 22% Completed |  5min 34.0s


KeyboardInterrupt: 

In [ ]:
%time len(df)

In [ ]:
# http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html
# turn those text labels into numerical
text_cols = ['UniqueCarrier', 'Origin', 'Dest']
le = preprocessing.LabelEncoder()
for c in text_cols:
    # print (c,set(df[c].values))
    flist = list(set(df[c].values))
    # print(flist)
    le.fit(flist)
    leo = le.transform(flist)
    # print (c,flist,leo)
    df[c+'_'] = df[c]
    df[c+'_'].replace(flist,value=leo,inplace=True)

In [ ]:
df['Cancelled'] = df.apply(lambda row: False if row['Cancelled'] == 0 else True, axis='columns')
df['Diverted'] = df.apply(lambda row: False if row['Diverted'] == 0 else True, axis='columns')

In [ ]:
df['@timestamp'] = df.apply(lambda row: pd.Timestamp('%s-%s-%s;%04d'%(row['Year'], row['Month'], row['DayofMonth'], row['CRSDepTime'])),axis='columns')

In [ ]:
df.head()

In [ ]:
timestamps = df['@timestamp']